In [9]:
from raco.backends.spark.connection import SparkConnection
from raco.backends.spark.catalog import SparkCatalog
from raco.backends.spark.algebra import SparkAlgebra
from raco.backends.myria.connection import MyriaConnection
from raco.backends.myria.catalog import MyriaCatalog
from raco.backends.myria import MyriaLeftDeepTreeAlgebra
from raco.backends.federated.connection import FederatedConnection
from raco.backends.federated.catalog import FederatedCatalog
from raco.backends.federated import FederatedAlgebra
from raco.backends.federated.algebra import FederatedExec

import raco.myrial.interpreter as interpreter
import raco.myrial.parser as myrialparser
from optparse import OptionParser

import raco.viz

import os

In [10]:
def get_myria_connection():
    rest_url = 'https://rest.myria.cs.washington.edu:1776'
    execution_url = 'http://demo.myria.cs.washington.edu'
    connection = MyriaConnection(rest_url=rest_url,
                                 execution_url=execution_url)
    return connection


In [11]:
def get_spark_connection():
    connection = SparkConnection('local')
    return connection

In [12]:
def query_spark(myriaconnection, sparkconnection, program):
    myrial_code = program

    myriacatalog = MyriaCatalog(myriaconnection)
    sparkcatalog = SparkCatalog(sparkconnection)

    catalog = FederatedCatalog([myriacatalog, sparkcatalog])
    parser = myrialparser.Parser()
    processor = interpreter.StatementProcessor(catalog, True)
    statement_list = parser.parse(myrial_code)
    processor.evaluate(statement_list)

    algebras = [MyriaLeftDeepTreeAlgebra(), SparkAlgebra()]
    falg = FederatedAlgebra(algebras, catalog)
    
    pd = processor.get_physical_plan(target_alg=falg)
    sparkconnection.execute_query(pd.args[0].plan)


In [13]:
program_test_project_dimension ="""
abc = scan('/users/shrainik/downloads/sample.dat');
b = [from abc where abc.i>5 emit i, j ];
store(b, '/users/shrainik/downloads/sample-out.dat');
"""

In [14]:
myriaconn = get_myria_connection()
sparkconn = get_spark_connection()


ValueError: Cannot run multiple SparkContexts at once; existing SparkContext(app=pyspark-shell, master=local) created by __init__ at raco/backends/spark/connection.py:32 

In [15]:
query_spark(myriaconn, sparkconn, program_test_project_dimension)

[('i', 'LONG_TYPE'), ('j', 'LONG_TYPE'), ('value', 'LONG_TYPE')]
[('i', 'LONG_TYPE'), ('j', 'LONG_TYPE'), ('value', 'LONG_TYPE')]
AFTER Spark RULES
SparkStore(public:adhoc:/users/shrainik/downloads/sample-out.dat)[SparkProject(i,j)[SparkSelect((i > 5))[SparkScan(public:adhoc:/users/shrainik/downloads/sample.dat)]]]
dot version after spark rules
digraph G {
      ratio = "fill" ;
      size = "4.0, 4.0" ;
      page = "4, 4" ;
      margin = 0 ;
      mincross = 2.0 ;
      rankdir = "BT" ;
      nodesep = 0.25 ;
      ranksep = 0.25 ;
      node [fontname="Helvetica", fontsize=10,
            shape=oval, style=filled, fillcolor=white ] ;

      // The nodes
      "4509900112" [label="SparkStore(public:adhoc:/users/shrainik/downloads/sample-out.dat)"] ;
      "4509898960" [label="SparkProject(i,j)"] ;
      "4509898768" [label="SparkSelect((i > 5))"] ;
      "4509899984" [label="SparkScan(public:adhoc:/users/shrainik/downloads/sample.dat)"] ;

      // The edges
      "4509898960" -> "4